# Import and Initialization

In [ ]:
import os
import requests
import datetime as dt
import time
import webbrowser
import pandas as pd

gitIGNORE = 'gitIGNORE'
nasa_api = f'nasa-api.txt'

def createMyFolders(folder_names_list: list, destination: str):
    """
    desitination: absolute path
    """
    import os

    if not os.path.exists(destination): # check if destination exists
        print(f"Destination does NOT exists: {destination}")
        return
    
    # loop thru all folder names and attempt to create them
    for folder_name in folder_names_list:
        folder_path = os.path.join(destination, folder_name)
        try:
            if os.path.exists(folder_path):
                print(f"folder exists: {folder_path}")
            elif not os.path.exists(folder_path):
                os.makedirs(folder_path, exist_ok=True)
                print(f"Created folder: {folder_path}")
        except Exception as e:
            print(f"Failed to create folder {folder_path}: {e}")

    return None

createMyFolders([gitIGNORE], os.getcwd()) # create folders in the current working directory

with open(rf"{gitIGNORE}\{nasa_api}", 'r') as file:
    apikey = file.read()



folder exists: c:\NASA-open\gitIGNORE


# Image of the Day

In [39]:
# # image of the day, single request for today 

# baseURL = 'https://api.nasa.gov/planetary/apod'
# params = {
#     'api_key': apikey,
#     'date': '2025-08-16',  # comment out for automatically getting today's date
#     'hd': True             
# }

# response = requests.get(baseURL, params=params)

# if response.status_code == 200:
#     data = response.json()
#     print(f"Title: {data['title']}")
#     print(f"Date: {data['date']}")
#     exp = data['explanation'].split('. ')
#     [print(f'{_}. ') for _ in exp]  
#     mediaType = data['media_type']
#     print(f"File Type: {mediaType}")
#     print(f"Image URL: {data['hdurl'] if 'hdurl' in data else data['url']}")
# else:
#     print(f"Error {response.status_code}: {response.text}")


# Batch Requesting Multiple Historical Days Image of the Day

In [ ]:
# -------------------- USER INPUTS -------------------- #
today_date = dt.datetime.now().date().strftime("%Y-%d-%m")  # get today's date for logging the acitivity 
startDate = "2025-06-30"   # when we start to request image of the day
api_timeout_sec = 5        # use a time out to avoid/delay reaching api request limit
num_of_days =2        # going back from the startDate, how many days do we want to get image-of-the-day for
# ----------------------------------------------------- #



baseURL = "https://api.nasa.gov/planetary/apod"   
startDate = pd.to_datetime(startDate).date()    # convert string format of startDate to date object to allow subtraction/addition with pd.timedelta()

with open('log.txt', 'a', encoding='utf-8') as logFile: 

    for i in range(num_of_days):  # start the range form 0 so the start date can be included 
        reqDate = startDate - pd.Timedelta(days=i)  # starting with the startDate, calculate the prior date
        params = {'api_key': apikey, 'date':f'{reqDate}', 'hd':True} # request parameters
        response = requests.get(url=baseURL, params=params)

        data = response.json()    # parse the response into a jason

        title = data.get('title', 'no title')    
        explanation = data.get('explanation', 'None').split('. ')   # to avoid a long single line of description, .split() so we can print lines with list comprehension later
        mediaType = data.get('media_type', 'unknown')
        url = data.get('hdurl') if 'hdurl' in data else data.get('url', 'unknown')   # get hd url if it exists, if not, then get the regular url

        if url:    #check to make sure there is a url
            print('Request Date', reqDate)
            print('Title: ', title)
            print('url: ', url)
            print('Explanation:')
            [print(f"{_}.") for _ in explanation]
            
 
            if mediaType == 'image' or url.lower().endswith('.gif'):   # open the url if it's an image or a gif
                webbrowser.open_new_tab(url)

            # logging the description for the particular date
            logFile.write(f"Request Date: {reqDate}\n")
            logFile.write(f"Title: {title}\n")
            logFile.write(f"{url}\n")
            logFile.write('Explanation:\n')
            for _ in explanation:
                logFile.write(f'{_}.\n')


        else: 
            print("no URL available")
            logFile.write("no URL available")

        print('-----------------------------------------\n')
        logFile.write('-----------------------------------------\n')

        time.sleep(float(api_timeout_sec))  # avoid/delay api rate limit



Request Date 2025-06-30
Title:  NGC 4651: The Umbrella Galaxy
url:  https://apod.nasa.gov/apod/image/2506/UmbrellaGal_Alkuwari_3843.png
Explanation:
It's raining stars.
 What appears to be a giant cosmic umbrella is now known to be a tidal stream of stars stripped from a small satellite galaxy.
The main galaxy, spiral galaxy NGC 4651, is about the size of our Milky Way, while its stellar parasol appears to extend some 100 thousand light-years above this galaxy's bright disk.
A small galaxy was likely torn apart by repeated encounters as it swept back and forth on eccentric orbits through NGC 4651.
The remaining stars will surely fall back and become part of a combined larger galaxy over the next few million years.
The featured deep image was captured in long exposures from Saudi Arabia.
The Umbrella Galaxy lies about 50 million light-years distant toward the well-groomed northern constellation of Berenice's Hair (Coma Berenices).
  APOD in a Modern Format StellarSnap.
-----------------

KeyboardInterrupt: 